In [ ]:
import os
print(os.listdir("../input"))

Подключите необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)}); 
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy.stats import chi2_contingency, fisher_exact

Загрузите данные. Они находятся в директории input.

In [ ]:
df = pd.read_csv("../input/bank-additional-full.csv", sep = ';')
#После ознакомлением с датасетом, можно сделать вывод, что целевой (target) переменной является 'y',
#которая показывает, открыл ли клиент срочный депозит. Основная задача прогнозной модели - предсказать,
#откроет ли человек срочный депозит, в зависимости от имеющихся о нём данных, таких как, например, 
#его возраст, семейное положение, рабочая должность и тд.

In [ ]:
df.info()
#Выводим общую информацию о датасете. Так можно узнать тип каждого признака, а также есть ли в данных пропуски

In [ ]:
#Не все данные бывают коррекными, поэтому на примере столбца 'age' уберем "выбросы".
sns.boxplot(df['age'])
#Всё, что находится за "усиками", это выбросы - явно некорректные значения, которые будут мешать дальнейшей работе. 
#Их лучше удалить.

In [ ]:
def outliers_indices(feature):
    
    #Будем считать выбросами все точки, выходящие за пределы трёх сигм.
    
    mid = df[feature].mean()
    sigma = df[feature].std()
    return df[(df[feature] < mid - 3*sigma) | (df[feature] > mid + 3*sigma)].index

In [ ]:
wrong_age = outliers_indices('age')
out = set(wrong_age)

print(len(out))

In [ ]:
#Из набора данных будет удалено 369 объектов-выбросов, что не является существенным в данном случае.

In [ ]:
df.drop(out, inplace=True)

Каков средний возраст холостых/незамужних клиентов?

In [ ]:
#холостые
df[df['marital'] == 'single']['age'].mean()

In [ ]:
#в разводе 
df[df['marital'] == 'divorced']['age'].mean()

In [ ]:
#холостые и в разводе
df[(df['marital'] == 'single') | (df['marital'] == 'divorced')]['age'].mean()

В какой день недели (признак **day_of_week**) чаще всего звонили клиентам, отказавшимся от депозита?

In [ ]:
df[df['y'] == 'no'].groupby('day_of_week').size().idxmax()

Постройте инфографику по признакам **marital** и **y**. Какие выводы можно сделать?

In [ ]:
#1 график
data = {
        'открывшие депозит': df.loc[df['y'] == 'yes'].groupby('marital').size() ,
        'отказавшиеся от депозита': df.loc[df['y'] == 'no'].groupby('marital').size()
       }
dff = pd.DataFrame(data)
dff.plot(kind='bar')
plt.show()

#2 график
labels = 'divorced', 'married', 'single', 'unknown'
fig,ax=plt.subplots()
plt.ylabel('%')
ax.set_title("Процентное соотношение тех, кто взял депозит")
ax.bar(labels, df.loc[df['y'] == 'yes'].groupby('marital').size() * 100 /  df.groupby('marital').size())

#Посмотрев на первый график, можно сделать вывод о том, что наибольшее число людей, открывших депозит, это люди,
#состоящие в браке. Однако, если посмотреть на второй график, с процентным соотношением общего числа людей из каждой
#категории и теми, кто в итоге оформил депозит, то самая высокая колонка выходит в категории "unknown". Но её, по 
#понятным причинам, нельзя рассматривать как реальную "категорию". Тогда выходит, что чаще всего, срочные депозиты
#оформляют холостые/незамужние клиенты, а реже всего - люди в браке.

Имеется ли связь между наличием кредита (**default**) у клиента и результатом? Постройте инфографику.

In [ ]:
#Так как данные категориальные, воспользуемся коэффициентом хи-квадрат (chi-square test)
p = chi2_contingency(pd.crosstab(df['default'], df['y']))
print("p-value = " + str(p[1]))
#p-value < 0.05 => взаимосвязь (корреляция) между наличием кредита у клиента и результатом статистически значима.

In [ ]:
plt.figure(figsize=(7, 7))
sns.heatmap(pd.crosstab(df['default'], df['y']), fmt="d", cmap="PiYG", annot=True);

Визуализируйте информацию об уровне образования в зависимости от среднего возраста. Есть ли статистически значимая взаимосвязь между ними?

In [ ]:
df.groupby('education')['age'].mean().plot(kind='bar', color = 'purple')

In [ ]:
p = chi2_contingency(pd.crosstab(df['education'], df['age']))
print("p-value = " + str(p[1]))
#p-value < 0.05 => статистически значимая взаимосвязь есть.

Что можно сказать о связи между длительностью контакта (признак **duration**) и возрастом клиента? Найдите коэффициент корреляции.

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(df['duration'], df['age'])
#Судя только по графику, можно сказать, что дольше всех раговаривают люди в возрасте 25-60 лет.

In [ ]:
r = pearsonr(df['duration'], df['age'])
print('Pearson correlation:', r[0], 'p-value:', r[1])
#p-value > 0.05 => взаимосвязь (корреляция) между длительностью разговора и возрастом человека статистически незначима.

Как связаны признаки **education** и **housing**?

In [ ]:
plt.figure(figsize=(7, 7))
sns.heatmap(pd.crosstab(df['education'], df['housing']), fmt="d", cmap="PiYG", annot=True);

labels = ['basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate','professional.course', 
          'university.degree', 'unknown']
fig,ax=plt.subplots(figsize=(17, 7))
plt.ylabel('%')
#plt.figure())
ax.set_title("Процентное соотношение тех, у кого есть жилищный кредит")
ax.bar(labels, df.loc[df['housing'] == 'yes'].groupby('education').size() * 100 /  df.groupby('education').size())
#df.loc[df['housing'] == 'yes'].groupby('education').size() * 100 /  df.groupby('education').size()
#Судя по первому графику, больше всего жилищных кредитов у людей с "university.degree" и"high.school".
#Второй график это процентное соотношение общего количества людей каждой категории из столбца "education" 
#и наличием жилищного кредита. Можно видеть, что чаще всего он есть у людей из категории "professional.course",
#далее у "university.degree" и у "illiterate".

In [ ]:
print("p-value: ", chi2_contingency(pd.crosstab(df['education'], df['housing']))[1])
#p-value < 0.05 => связь статистически подтверждается.

Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

In [ ]:
#Построим матрицу корреляций Спирмена для числовых признаков и исследуем их влияние на "у"
df['y_bin']=(df['y']=='yes').astype(int)
numeric = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 
           'euribor3m', 'nr.employed', 'y_bin']
df[numeric].corr(method = 'spearman')

In [ ]:
#Визуализация таблицы
sns.heatmap(df[numeric].corr(method='spearman'))
#Чем "дальше" значение коэффициента корреляции находится от нуля, тем больше линейная или нелинейная взаимосвязь 
#между объектами.

In [ ]:
#Судя по графику, наибольшее влияние на то, откроет человек депозит или нет, оказывает переменная "duration". Зависимость
#между длительностью разговора и переменной "y" - линейная, а значит чем дольше человек говорит по телефону, тем больше
#вероятность, что он/она откроет депозит. Также небольшая линейная зависимость присутствует с "previous" (количество 
#звонков по поводу депозита от других компаний). Небольшая нелинейная зависимость с "y" есть с такими колонками как "pdays" 
#(количестводней,прошедших после того, как с клиентом в последний раз связывались из другой компании по поводу депозита),
#"euribor3m" (процентная ставка по вкладам в евро на период 3 месяца), "nr.employed" (number of employees - quarterly
#indicator). Нелинейная зависимость означает, что увеличение значения одной переменной, ведет к уменьшению значения 
#коррелирующей с ней переменной.